### Customers

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df_cust = spark.read.table("pysparkdbt.bronze.customers")

In [0]:
display(df_cust)

In [0]:
df_cust = df_cust.withColumn("domain", split(col("email"), "@").getItem(1))
display(df_cust)